# Aprendizaje automático I

In [167]:
import pandas as pd
def merge_data(bookings_file, hotels_file, output_file):

    df_book = pd.read_csv(bookings_file)
    df_hotel = pd.read_csv(hotels_file)
    df = pd.merge(df_book, df_hotel, on='hotel_id')
    df.to_csv(output_file, index=False)
    print(f"Datos cargados y guardados en: {output_file}")

# 1. Cargar y fusionar datos
merge_data('data/bookings_train.csv', 'data/hotels.csv', 'data/data.csv')

Datos cargados y guardados en: data/data.csv


In [168]:
def split_train_validation(df, train_cutoff_date, validation_output_filename):
    """
    Divide el DataFrame en conjuntos de entrenamiento y validación y guarda el DataFrame de validación en un archivo CSV.
    """
    df_validation = df[df["reservation_status"] == "Booked"].copy()
    df_train = df[df["reservation_status"] != "Booked"].copy()

    df_train = df_train[df_train['booking_date'] <= train_cutoff_date].copy()
    df_validation = df_validation[df_validation['booking_date'] > train_cutoff_date].copy()

    # Guardar df_validation en un archivo CSV
    df_validation.to_csv(validation_output_filename, index=False)
    print(f"El DataFrame df_validation se ha guardado en '{validation_output_filename}'.")

    print("Se han creado los DataFrames df_train y df_validation.")

    return df_train, df_validation


df = pd.read_csv('data/data.csv')
df_train, df_validation = split_train_validation(df, '2016-06-30', 'data/data_validation.csv')

El DataFrame df_validation se ha guardado en 'data/data_validation.csv'.
Se han creado los DataFrames df_train y df_validation.


In [146]:
df_validation

,board,country_x,market_segment,distribution_channel,room_type,required_car_parking_spaces,special_requests,stay_nights,rate,total_guests,...,booking_date,reservation_status_date,reservation_status,hotel_type,country_y,parking,total_rooms,restaurant,pool_and_spa,avg_review
31,BB,SPA,Corporate,Corporate,A,NaN,1.0,1.0,67.00,3.0,...,2017-08-20,2017-08-20,Booked,City Hotel,SPA,False,126,True,False,4.75
756,BB,SPA,Corporate,Corporate,A,NaN,2.0,1.0,67.00,3.0,...,2017-08-22,2017-08-22,Booked,City Hotel,SPA,False,126,True,False,4.75
898,BB,SPA,Corporate,Corporate,A,0.0,1.0,1.0,65.00,3.0,...,2017-07-14,2017-07-14,Booked,City Hotel,SPA,False,126,True,False,4.75
899,BB,SPA,Corporate,Corporate,A,0.0,1.0,3.0,195.00,3.0,...,2017-07-01,2017-07-01,Booked,City Hotel,SPA,False,126,True,False,4.75
993,BB,SPA,Corporate,Corporate,A,NaN,2.0,1.0,80.00,3.0,...,2017-08-26,2017-08-26,Booked,City Hotel,SPA,False,126,True,False,4.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49188,HB,POR,Online TA,TA/TO,A,0.0,1.0,4.0,773.60,4.0,...,2017-07-27,2017-07-27,Booked,Resort Hotel,SPA,False,45,True,False,4.54
49192,BB,SWE,Online TA,TA/TO,D,NaN,2.0,5.0,1125.00,4.0,...,2017-07-20,2017-07-20,Booked,Resort Hotel,SPA,False,45,True,False,4.54
49194,BB,IRL,Online TA,TA/TO,A,0.0,1.0,4.0,743.96,4.0,...,2017-07-01,2017-07-01,Booked,Resort Hotel,SPA,False,45,True,False,4.54
50085,HB,SPA,Online TA,TA/TO,A,0.0,2.0,1.0,231.00,4.0,...,2017-08-16,2017-08-16,Booked,Resort Hotel,SPA,False,45,True,False,4.54


In [170]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


def preprocess_train_data(df_train, output_filename='data_model.csv'):  # Añadido output_filename
    """
    Preprocesa el DataFrame de entrenamiento.
    """
    # Eliminar hotel_id y country_x y days_diff
    df_train.drop(columns=['hotel_id', 'country_x', 'days_diff'], errors='ignore', inplace=True)

    # Renombrar la columna 'country_y' a 'country'
    if 'country_y' in df_train.columns:
        df_train.rename(columns={'country_y': 'country'}, inplace=True)

    # 2.1 Transformaciones de fechas y creación de variable target (solo para df_train)
    df_train['booking_date'] = pd.to_datetime(df_train['booking_date'])
    df_train['arrival_date'] = pd.to_datetime(df_train['arrival_date'])
    df_train['reservation_status_date'] = pd.to_datetime(df_train['reservation_status_date'])

    df_train["days_diff"] = (df_train["arrival_date"] - df_train["reservation_status_date"]).dt.days
    df_train["cancelled_last_30_days"] = ((df_train["reservation_status"] == "Canceled") & (df_train["days_diff"] <= 30)).astype(int)

    # 6. Cálculo de la anticipación de la reserva (solo para df_train )
    df_train['advance_reservation_days'] = (df_train['arrival_date'] - df_train['booking_date']).dt.days

    # De boleano a entero (solo para df_train)
    df_train['parking'] = df_train['parking'].astype(int)
    df_train['restaurant'] = df_train['restaurant'].astype(int)
    df_train['pool_and_spa'] = df_train['pool_and_spa'].astype(int)

    # 3. Manejo de outliers en 'rate' y 'total_guests' usando el rango intercuartil (IQR) solo en df_train (ANTES DE LA NORMALIZACIÓN)
    q1_rate = df_train["rate"].quantile(0.25)
    q3_rate = df_train["rate"].quantile(0.75)
    iqr_rate = q3_rate - q1_rate
    lower_bound_rate = q1_rate - 1.5 * iqr_rate
    upper_bound_rate = q3_rate + 1.5 * iqr_rate
    df_train = df_train[(df_train["rate"] >= lower_bound_rate) & (df_train["rate"] <= upper_bound_rate)]

    q1_guests = df_train["total_guests"].quantile(0.25)
    q3_guests = df_train["total_guests"].quantile(0.75)
    iqr_guests = q3_guests - q1_guests
    lower_bound_guests = q1_guests - 1.5 * iqr_guests
    upper_bound_guests = q3_guests + 1.5 * iqr_guests
    df_train = df_train[(df_train["total_guests"] >= lower_bound_guests) & (df_train["total_guests"] <= upper_bound_guests)]

    # 2.1 Separación de Columnas Numéricas y Categóricas (Solo para df_train)
    numerical_cols = df_train.select_dtypes(include=np.number).columns.tolist()
    categorical_cols = df_train.select_dtypes(include='object').columns.tolist()

    # Eliminar columnas de fecha antes de ColumnTransformer
    date_cols = ['booking_date', 'arrival_date', 'reservation_status_date']
    df_train = df_train.drop(columns=date_cols)

    # Imputar valores nulos específicos (solo para df_train)
    df_train['special_requests'].fillna(0, inplace=True)
    df_train['required_car_parking_spaces'].fillna(0, inplace=True)

    # Elimino columnas redundantes reservation_status (solo para df_train)
    df_train.drop(columns=['reservation_status'], inplace=True)

    # Actualizar categorical_cols después de eliminar 'reservation_status' (solo para df_train)
    categorical_cols = [col for col in categorical_cols if col != 'reservation_status']

    # Separar columnas numericas que no son 0 y 1 para normalizar (solo para df_train)
    cols_to_normalize = []
    for col in numerical_cols:
        if len(df_train[col].unique()) > 2:
            cols_to_normalize.append(col)

    # Pipelines para transformaciones numéricas y categóricas (solo para df_train)
    numerical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ])

    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
    ])

    # ColumnTransformer (solo para df_train)
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, cols_to_normalize),
            ('cat', categorical_transformer, categorical_cols)
        ],
        remainder='passthrough'
    )

    # Aplicar transformaciones (solo para df_train)
    transformed_data = preprocessor.fit_transform(df_train)

    # Obtener nombres de las columnas transformadas (solo para df_train)
    numerical_cols_transformed = [col for col in cols_to_normalize]
    categorical_cols_transformed = preprocessor.named_transformers_['cat']['onehot'].get_feature_names_out(categorical_cols).tolist()
    remaining_cols = [col for col in df_train.columns if col not in cols_to_normalize + categorical_cols]

    transformed_columns = numerical_cols_transformed + categorical_cols_transformed + remaining_cols

    df_model = pd.DataFrame(transformed_data, columns=transformed_columns)

    # Redondeo y conversión a entero (excepto 'avg_review') (solo para df_train)
    cols_to_convert = [col for col in df_model.columns if col in numerical_cols and col not in ['avg_review']]
    for col in cols_to_convert:
        if col in df_model.columns:
            df_model[col] = df_model[col].round().astype(int)

    # Eliminar columnas con baja correlación (solo para df_train)
    correlation_threshold = 0.02
    correlations = df_model.corr()['cancelled_last_30_days'].abs()
    cols_to_drop = correlations[correlations < correlation_threshold].index.tolist()
    df_model.drop(columns=cols_to_drop, inplace=True)

    # Guardar el DataFrame en un archivo CSV
    df_model.to_csv(output_filename, index=False)
    print(f"El DataFrame df_model se ha guardado en '{output_filename}'")

    return df_model

# Ejemplo de uso:
df_model = preprocess_train_data(df_train) #ruta cambiada

El DataFrame df_model se ha guardado en 'data_model.csv'
